In [22]:
require(MASS)

In [23]:
# load data from file
data = read.csv("~/research_data/mozilla_voice_comparison/ppt_data.csv")
wpm = read.csv("~/research_data/mozilla_voice_comparison/voice_metadata.csv")

data = as.data.frame(lapply(data, function (x) if (is.factor(x)) factor(x) else x))

In [24]:
data = subset(data, voice != 'Text only')
data$rate.speed = ordered(data$rate.speed, levels = c("Much too slow","Too slow","Just right","Too fast","Much too fast"))
data$rate.speed.simple = data$rate.speed
levels(data$rate.speed.simple) = c('Very poor','poor','Just right','poor','Very poor')
data$rate.speed.simple = ordered(data$rate.speed.simple, levels = c('Very poor','poor','Just right'))
data$rate.experience = ordered(data$rate.experience, levels = c("Very poor", "Poor", "OK", "Good", "Excellent"))
data$gender = factor(data$gender, levels = c('Woman','Man'), labels = c('Female','Male'))
data = subset(data, gender %in% c('Male','Female'))
                       
wpm$TTS.Voice.Gender = factor(wpm$TTS.Voice.Gender)
wpm$Type = factor(wpm$Type)

data = merge(data, wpm, by.x = 'voice', by.y = 'Voice')

In [10]:
data <- within(data, voice <- relevel(voice, ref = 3))

In [18]:
m <- polr(rate.experience ~ WPM + Type + gender + TTS.Voice.Gender + headphones + TTS.Voice.Gender*gender, data = data, Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable$odds.ratio = exp(ctable$Value)
ctable

Call:
polr(formula = rate.experience ~ WPM + Type + gender + TTS.Voice.Gender + 
    headphones + TTS.Voice.Gender * gender, data = data, Hess = TRUE)

Coefficients:
                                    Value Std. Error t value
WPM                             -0.004071   0.003489 -1.1667
TypeTTS                         -0.724650   0.167546 -4.3251
genderMale                      -0.017034   0.143544 -0.1187
TTS.Voice.GenderMale             0.429553   0.193131  2.2242
headphonesYes                    0.393150   0.130909  3.0032
genderMale:TTS.Voice.GenderMale -0.300458   0.245726 -1.2227

Intercepts:
               Value   Std. Error t value
Very poor|Poor -4.7245  0.6485    -7.2854
Poor|OK        -2.6765  0.6209    -4.3110
OK|Good        -1.0263  0.6162    -1.6655
Good|Excellent  0.8030  0.6156     1.3044

Residual Deviance: 2772.006 
AIC: 2792.006 

[1] "0.243*"
[1] "0.000*"
[1] "0.906*"
[1] "0.026*"
[1] "0.003*"
[1] "0.221*"
[1] "0.000*"
[1] "0.000*"
[1] "0.096*"
[1] "0.192*"


,Value,Std. Error,t value,pvalue,sig,odds.ratio
,<dbl>,<dbl>,<dbl>,<dbl>,<list>,<dbl>
WPM,-0.004071013,0.003489215,-1.1667418,0.243,0.243,0.995937262
TypeTTS,-0.724649505,0.167546068,-4.3250762,0.000,0.000*,0.484494352
genderMale,-0.017034303,0.143544354,-0.1186693,0.906,0.906,0.983109960
TTS.Voice.GenderMale,0.429552778,0.193130886,2.2241537,0.026,0.026*,1.536570183
headphonesYes,0.393149898,0.130909143,3.0032272,0.003,0.003*,1.481640467
genderMale:TTS.Voice.GenderMale,-0.300457639,0.245725954,-1.2227346,0.221,0.221,0.740479271
Very poor|Poor,-4.724480211,0.648484181,-7.2854209,0.000,0.000*,0.008875326
Poor|OK,-2.676491589,0.620852822,-4.3109921,0.000,0.000*,0.068804124
OK|Good,-1.026287872,0.616199093,-1.6655134,0.096,0.096,0.358334679


In [ ]:
m <- polr(rate.experience ~ poly(WPM,2) + voice + Type + gender + headphones + voice*gender, data = data, Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable
#m <- polr(rate.experience ~ TTS.Voice.Gender, data = data, Hess=TRUE)

In [25]:
m <- polr(rate.experience ~ poly(WPM,2) + Type + gender + TTS.Voice.Gender + headphones + TTS.Voice.Gender*gender, data = data, Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable$odds.ratio = exp(ctable$Value)
ctable

Call:
polr(formula = rate.experience ~ poly(WPM, 2) + Type + gender + 
    TTS.Voice.Gender + headphones + TTS.Voice.Gender * gender, 
    data = data, Hess = TRUE)

Coefficients:
                                    Value Std. Error  t value
poly(WPM, 2)1                   -2.088792     1.8291 -1.14195
poly(WPM, 2)2                   -3.638974     1.8367 -1.98128
TypeTTS                         -0.810601     0.1734 -4.67411
genderMale                      -0.004877     0.1436 -0.03396
TTS.Voice.GenderMale             0.432191     0.1931  2.23846
headphonesYes                    0.391553     0.1308  2.99343
genderMale:TTS.Voice.GenderMale -0.309112     0.2457 -1.25801

Intercepts:
               Value    Std. Error t value 
Very poor|Poor  -4.1113   0.2953   -13.9205
Poor|OK         -2.0620   0.2270    -9.0833
OK|Good         -0.4054   0.2173    -1.8653
Good|Excellent   1.4281   0.2231     6.4012

Residual Deviance: 2768.067 
AIC: 2790.067 

[1] "0.253*"
[1] "0.048*"
[1] "0.000*"
[1] "0.973*"
[1] "0.025*"
[1] "0.003*"
[1] "0.208*"
[1] "0.000*"
[1] "0.000*"
[1] "0.062*"
[1] "0.000*"


,Value,Std. Error,t value,pvalue,sig,odds.ratio
,<dbl>,<dbl>,<dbl>,<dbl>,<list>,<dbl>
"poly(WPM, 2)1",-2.088791568,1.8291381,-1.14195400,2.534731e-01,0.253,0.12383669
"poly(WPM, 2)2",-3.638973530,1.8366783,-1.98127978,4.755991e-02,0.048*,0.02627931
TypeTTS,-0.810601432,0.1734239,-4.67410535,2.952378e-06,0.000*,0.44459059
genderMale,-0.004876927,0.1436051,-0.03396067,9.729085e-01,0.973,0.99513495
TTS.Voice.GenderMale,0.432191104,0.1930749,2.23846297,2.519088e-02,0.025*,1.54062951
headphonesYes,0.391552686,0.1308039,2.99343276,2.758583e-03,0.003*,1.47927586
genderMale:TTS.Voice.GenderMale,-0.309111981,0.2457147,-1.25801158,2.083876e-01,0.208,0.73409856
Very poor|Poor,-4.111289357,0.2953414,-13.92046510,4.757978e-44,0.000*,0.01638663
Poor|OK,-2.061999595,0.2270095,-9.08331994,1.053125e-19,0.000*,0.12719937


In [ ]:
vars = c(
    'poly(WPM, 2)1' = 'WPM',
    'poly(WPM, 2)2' = 'WPM^2',
    'TypeTTS' = 'Type_Voice',
    'genderMale' = 'Gender_Listener',
    'TTS.Voice.GenderMale' = 'Gender_Voice',
    'headphonesYes' = 'Headphones_Listener',
    'genderMale:TTS.Voice.GenderMale' = 'Gender*Gender',
    'Very poor|Poor' = 'Very poor|Poor',
    'Poor|OK' = 'Poor|OK',
    'OK|Good' ='OK|Good',
    'Good|Excellent' = 'Good|Excellent'
)
ctable = merge(ctable,vars, by.x=0, by.y=0)
ctable

In [12]:
addSig = function (x) {
    print(paste(sprintf(x, fmt = '%#.3f'),'*',sep=""))
    if (x < .05) {return(paste(sprintf(x, fmt = '%#.3f'),'*',sep=""))}
    else {return(sprintf(x, fmt = '%#.3f'))}
}

In [ ]:
chisq.test(data$TTS.Voice.Gender, data$voice)

In [ ]:
m <- polr(rate.speed ~ poly(WPM,2) + voice + Type + gender + headphones + voice*gender, data = data, Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable
#m <- polr(rate.experience ~ TTS.Voice.Gender, data = data, Hess=TRUE)

In [19]:
m <- polr(rate.speed ~ poly(WPM,2) + Type + gender + TTS.Voice.Gender + headphones + TTS.Voice.Gender*gender, data = data, Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable

Call:
polr(formula = rate.speed ~ poly(WPM, 2) + Type + gender + TTS.Voice.Gender + 
    headphones + TTS.Voice.Gender * gender, data = data, Hess = TRUE)

Coefficients:
                                   Value Std. Error t value
poly(WPM, 2)1                   14.29558     2.2564  6.3354
poly(WPM, 2)2                    5.60472     2.1752  2.5766
TypeTTS                          0.28195     0.2078  1.3568
genderMale                       0.11191     0.1743  0.6422
TTS.Voice.GenderMale             0.04755     0.2337  0.2035
headphonesYes                   -0.06177     0.1569 -0.3936
genderMale:TTS.Voice.GenderMale -0.37975     0.2961 -1.2827

Intercepts:
                       Value    Std. Error t value 
Much too slow|Too slow  -3.6829   0.3288   -11.2024
Too slow|Just right     -1.6727   0.2663    -6.2813
Just right|Too fast      2.0921   0.2701     7.7457
Too fast|Much too fast   5.4806   0.4846    11.3102

Residual Deviance: 1749.081 
AIC: 1771.081 

[1] "0.000*"
[1] "0.010*"
[1] "0.175*"
[1] "0.521*"
[1] "0.839*"
[1] "0.694*"
[1] "0.200*"
[1] "0.000*"
[1] "0.000*"
[1] "0.000*"
[1] "0.000*"


,Value,Std. Error,t value,pvalue,sig
,<dbl>,<dbl>,<dbl>,<dbl>,<list>
"poly(WPM, 2)1",14.29558497,2.2564459,6.3354433,0.000,0.000*
"poly(WPM, 2)2",5.60472166,2.1752327,2.5766079,0.010,0.010*
TypeTTS,0.28195149,0.2078108,1.3567700,0.175,0.175
genderMale,0.11191233,0.1742724,0.6421689,0.521,0.521
TTS.Voice.GenderMale,0.04755137,0.2336556,0.2035105,0.839,0.839
headphonesYes,-0.06176715,0.1569365,-0.3935806,0.694,0.694
genderMale:TTS.Voice.GenderMale,-0.37974860,0.2960501,-1.2827172,0.200,0.200
Much too slow|Too slow,-3.68292208,0.3287616,-11.2024106,0.000,0.000*
Too slow|Just right,-1.67266865,0.2662944,-6.2812762,0.000,0.000*


In [ ]:
m <- polr(rate.speed ~ poly(WPM, 2), data = subset(data), Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable

In [ ]:
m <- polr(rate.speed.simple ~ WPM, data = subset(data), Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
ctable

In [17]:
m <- polr(rate.speed.simple ~ poly(WPM, 2), data = subset(data), Hess=TRUE)
summary(m)
ctable <- coef(summary(m))
p <- round(pnorm(abs(ctable[, "t value"]), lower.tail = FALSE) * 2, digits = 3)
ctable <- cbind(ctable, "pvalue" = p)
ctable = as.data.frame.matrix(ctable)
ctable$sig = lapply(ctable$pvalue, addSig)
result = ctable

vars = c(
    'poly(WPM, 2)1' = '$\\mathit{WPM}',
    'poly(WPM, 2)2' = '$\\mathit{WPM^2}',
    'Very poor|poor' = '$\\mathit{Very.poor|Poor}',
    'poor|Just right' = '$\\mathit{poor|Just.right}'
)

rank = c(
    'poly(WPM, 2)1' = 1,
    'poly(WPM, 2)2' = 2,
    'Very poor|poor' = 3,
    'poor|Just right' = 4
)

result
result = merge(result,vars, by.x=0, by.y=0)
result = merge(result,rank, by.x='Row.names', by.y=0)
result

Call:
polr(formula = rate.speed.simple ~ poly(WPM, 2), data = subset(data), 
    Hess = TRUE)

Coefficients:
                Value Std. Error t value
poly(WPM, 2)1 -0.7728      2.130 -0.3629
poly(WPM, 2)2 -4.8599      2.126 -2.2858

Intercepts:
                Value    Std. Error t value 
Very poor|poor   -3.5458   0.1887   -18.7931
poor|Just right  -0.9211   0.0696   -13.2347

Residual Deviance: 1405.837 
AIC: 1413.837 

[1] "0.717*"
[1] "0.022*"
[1] "0.000*"
[1] "0.000*"


,Value,Std. Error,t value,pvalue,sig
,<dbl>,<dbl>,<dbl>,<dbl>,<list>
"poly(WPM, 2)1",-0.7728308,2.12967123,-0.3628874,0.717,0.717
"poly(WPM, 2)2",-4.8598728,2.12609659,-2.2858194,0.022,0.022*
Very poor|poor,-3.5457927,0.18867550,-18.7930744,0.000,0.000*
poor|Just right,-0.9211017,0.06959722,-13.2347485,0.000,0.000*


Row.names,Value,Std. Error,t value,pvalue,sig,y.x,y.y
<I<chr>>,<dbl>,<dbl>,<dbl>,<dbl>,<list>,<fct>,<dbl>
"poly(WPM, 2)1",-0.7728308,2.12967123,-0.3628874,0.717,0.717,$\mathit{WPM},1
"poly(WPM, 2)2",-4.8598728,2.12609659,-2.2858194,0.022,0.022*,$\mathit{WPM^2},2
poor|Just right,-0.9211017,0.06959722,-13.2347485,0.000,0.000*,$\mathit{poor|Just.right},4
Very poor|poor,-3.5457927,0.18867550,-18.7930744,0.000,0.000*,$\mathit{Very.poor|Poor},3
